In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
path_root = '/home/dmr/DatasetForExperiments/ransomware/'

In [2]:
from keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=path_root, target_size=(64,64), batch_size=10000)

2023-05-02 17:08:33.386934: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Found 101 images belonging to 10 classes.


In [3]:
# Print information about Images 
imgs, labels = next(batches)
imgs.shape # Images with width x length x depth
labels.shape # Lablels with batch_size, number of classes

(101, 10)

In [4]:
import numpy as np
import scipy as sp
from PIL import Image

# Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.1)

In [5]:
X_train.shape

(90, 64, 64, 3)

In [6]:
X_test.shape

(11, 64, 64, 3)

In [7]:
y_train.shape

(90, 10)

In [8]:
y_test.shape

(11, 10)

In [9]:
# CNN Model

In [10]:
import keras
import tensorflow
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import AUC

In [11]:
num_classes = 10

In [12]:
def malware_model():
    Malware_model = Sequential()
    # Convolutional Layer : 30 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(31, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))

    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Convolutional Layer : 15 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(17, (3, 3), activation='relu'))
    
    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # DropOut Layer : Dropping 25% of neurons.
    Malware_model.add(Dropout(0.25))
    
    # Flatten Layer
    Malware_model.add(Flatten())
    
    # Dense/Fully Connected Layer : 128 Neurons, Relu activation function
    Malware_model.add(Dense(128, activation='relu'))
    
   # Malware_model.add(Dense(64, activation='relu'))
    
    # DropOut Layer : Dropping 50% of neurons.
    Malware_model.add(Dropout(0.5))
    
    # Dense/Fully Connected Layer : 50 Neurons, Softmax activation function
    Malware_model.add(Dense(50, activation='relu'))
    
    # Dense/Fully Connected Layer : num_class Neurons, Softmax activation function
    Malware_model.add(Dense(num_classes, activation='softmax'))
    
    
    Malware_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return Malware_model


    # sparse_categorical_cross_entropy
    # categorical_crossentropy
    

In [13]:
from keras import backend as K

def recall_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_test, y_pred):
    precision = precision_m(y_test, y_pred)
    recall = recall_m(y_test, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
Malware_model = malware_model()

2023-05-02 17:08:35.114442: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-02 17:08:35.373961: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-02 17:08:35.373980: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-05-02 17:08:35.374288: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 17:08:35.378409: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz
2023-05-02 17:08:35.3791

In [15]:
Malware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 31)        868       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 31)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 17)        4760      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 17)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 17)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3332)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [16]:
y_train.shape

(90, 10)

In [17]:
y_train_new = np.argmax(y_train, axis=1)

In [18]:
y_train_new

array([9, 4, 9, 2, 6, 7, 3, 0, 0, 3, 3, 1, 3, 2, 8, 4, 6, 7, 1, 1, 2, 6,
       0, 7, 2, 5, 8, 5, 0, 8, 8, 7, 5, 1, 3, 5, 1, 8, 9, 7, 9, 4, 4, 6,
       6, 9, 5, 7, 2, 3, 5, 2, 7, 2, 3, 0, 4, 9, 9, 0, 0, 8, 1, 3, 2, 5,
       5, 9, 6, 3, 7, 1, 1, 6, 6, 8, 7, 4, 8, 8, 8, 8, 0, 6, 5, 2, 7, 9,
       9, 4])

In [19]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_train_new),
                                                 y = y_train_new)

class_weights = {l:c for l,c in zip(np.unique(y_train_new), class_weights)}

In [20]:
Malware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m,precision_m, recall_m,AUC()])

In [21]:
Malware_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,  class_weight=class_weights)

2023-05-02 17:08:35.531782: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
3/3 [==============================] - 1s 267ms/step - loss: 0.6764 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4335 - val_loss: 0.5190 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.3990
Epoch 2/100
3/3 [==============================] - 0s 52ms/step - loss: 0.5300 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4844 - val_loss: 0.4339 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.3719
Epoch 3/100
3/3 [==============================] - 0s 51ms/step - loss: 0.4618 - f1_m: 0.0301 - precision_m: 0.1354 - recall_m: 0.0169 - auc: 0.4960 - val_loss: 0.3842 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.4596
Epoch 4/100
3/3 [==============================] - 0s 53ms/step - loss: 0.4363 - f1_m: 0.0158 - precision_m: 0.0583 - recall_m: 0.0091 - auc: 0.4864 - val_loss: 0.3616 - val_f1_

3/3 [==============================] - 0s 57ms/step - loss: 0.2154 - f1_m: 0.5034 - precision_m: 0.8555 - recall_m: 0.3600 - auc: 0.9196 - val_loss: 0.2259 - val_f1_m: 0.3077 - val_precision_m: 1.0000 - val_recall_m: 0.1818 - val_auc: 0.8843
Epoch 32/100
3/3 [==============================] - 0s 57ms/step - loss: 0.2230 - f1_m: 0.4242 - precision_m: 0.8125 - recall_m: 0.2881 - auc: 0.9013 - val_loss: 0.2209 - val_f1_m: 0.3077 - val_precision_m: 1.0000 - val_recall_m: 0.1818 - val_auc: 0.8792
Epoch 33/100
3/3 [==============================] - 0s 61ms/step - loss: 0.2006 - f1_m: 0.4871 - precision_m: 0.8702 - recall_m: 0.3391 - auc: 0.9535 - val_loss: 0.2239 - val_f1_m: 0.3077 - val_precision_m: 1.0000 - val_recall_m: 0.1818 - val_auc: 0.8921
Epoch 34/100
3/3 [==============================] - 0s 58ms/step - loss: 0.1956 - f1_m: 0.5756 - precision_m: 0.8677 - recall_m: 0.4314 - auc: 0.9426 - val_loss: 0.2138 - val_f1_m: 0.3077 - val_precision_m: 1.0000 - val_recall_m: 0.1818 - val_auc: 

3/3 [==============================] - 0s 54ms/step - loss: 0.0513 - f1_m: 0.9349 - precision_m: 0.9497 - recall_m: 0.9210 - auc: 0.9982 - val_loss: 0.1997 - val_f1_m: 0.7619 - val_precision_m: 0.8000 - val_recall_m: 0.7273 - val_auc: 0.8714
Epoch 64/100
3/3 [==============================] - 0s 54ms/step - loss: 0.0461 - f1_m: 0.9726 - precision_m: 0.9903 - recall_m: 0.9557 - auc: 0.9994 - val_loss: 0.1928 - val_f1_m: 0.7619 - val_precision_m: 0.8000 - val_recall_m: 0.7273 - val_auc: 0.8806
Epoch 65/100
3/3 [==============================] - 0s 53ms/step - loss: 0.0565 - f1_m: 0.9321 - precision_m: 0.9583 - recall_m: 0.9081 - auc: 0.9983 - val_loss: 0.2096 - val_f1_m: 0.7273 - val_precision_m: 0.7273 - val_recall_m: 0.7273 - val_auc: 0.8737
Epoch 66/100
3/3 [==============================] - 0s 52ms/step - loss: 0.0642 - f1_m: 0.9431 - precision_m: 0.9556 - recall_m: 0.9310 - auc: 0.9950 - val_loss: 0.2132 - val_f1_m: 0.7273 - val_precision_m: 0.7273 - val_recall_m: 0.7273 - val_auc: 

3/3 [==============================] - 0s 52ms/step - loss: 0.0250 - f1_m: 0.9523 - precision_m: 0.9567 - recall_m: 0.9479 - auc: 0.9996 - val_loss: 0.3121 - val_f1_m: 0.7619 - val_precision_m: 0.8000 - val_recall_m: 0.7273 - val_auc: 0.8747
Epoch 96/100
3/3 [==============================] - 0s 51ms/step - loss: 0.0314 - f1_m: 0.9661 - precision_m: 0.9661 - recall_m: 0.9661 - auc: 0.9998 - val_loss: 0.3232 - val_f1_m: 0.7619 - val_precision_m: 0.8000 - val_recall_m: 0.7273 - val_auc: 0.8365
Epoch 97/100
3/3 [==============================] - 0s 51ms/step - loss: 0.0178 - f1_m: 0.9785 - precision_m: 0.9818 - recall_m: 0.9754 - auc: 0.9999 - val_loss: 0.2985 - val_f1_m: 0.7619 - val_precision_m: 0.8000 - val_recall_m: 0.7273 - val_auc: 0.8333
Epoch 98/100
3/3 [==============================] - 0s 56ms/step - loss: 0.0250 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - auc: 1.0000 - val_loss: 0.2615 - val_f1_m: 0.7273 - val_precision_m: 0.7273 - val_recall_m: 0.7273 - val_auc: 

In [22]:
scores = Malware_model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 20ms/step - loss: 0.2398 - f1_m: 0.7619 - precision_m: 0.8000 - recall_m: 0.7273 - auc: 0.8701


In [23]:
loss, accuracy, f1_score, precision, recall = Malware_model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 20ms/step - loss: 0.2398 - f1_m: 0.7619 - precision_m: 0.8000 - recall_m: 0.7273 - auc: 0.8701


In [24]:
print('Final CNN accuracy: ', scores[1])

Final CNN accuracy:  0.7619047164916992
